In [77]:
import pandas as pd
import numpy as np
from distance_benchmark import compute_pfipf

In [18]:
org_cell = np.load("../data/testing_data/output_org_osaka20/cell_traj.npy")
ref_cell = np.load("../data/testing_data/output_ref_osaka20/cell_traj.npy")

In [189]:
fun_exp = [1, 1, 2, 3]
fun_exp_pfipf = [1, 1, 2, 3]

In [144]:
def mv_cell(cell_number, x, y):
    x_pos = cell_number % 32
    x_pos = 32 if x_pos == 0 else x_pos
    y_pos = int((cell_number - 1) / 32)
    
    #assert 1 <= x_pos + x < 33, "depassing x limits"
    #assert 0 <= y_pos + y < 32, "depassing y limits"
    if not ((1 <= x_pos + x < 33)) or not((0 <= y_pos + y < 32)):
        return cell_number
    
    x_pos += x
    y_pos += y
    
    return x_pos + 32*(y_pos)

In [145]:
def freq_pos_pers(array_pos):
    pos, frq = np.unique(array_pos, return_counts=True)
    frq = frq/400
    
    return pos, frq

In [146]:
from tqdm import tqdm

def anonymize(array):
    
    array = array.copy()
    
    for i in tqdm(range(array.shape[0])):
        pos, freq = freq_pos_pers(array[i])
        x = 0
        y = 0
        while (x == 0) and (y == 0):
            x = np.random.randint(-1,2)
            y = np.random.randint(-1,2)
        #stats = pd.DataFrame(freq).describe().T
        
        factor = freq.copy()
        first_perc = (int(freq.shape[0]*0.01)) or 1
        second_perc = (int(freq.shape[0]*0.04)) + first_perc
        third_perc = (int(freq.shape[0]*0.25)) + second_perc
        factor[np.argsort(freq)[-first_perc:]] = fun_exp[-1]
        factor[np.argsort(freq)[-second_perc:-first_perc]] = fun_exp[-2]
        factor[np.argsort(freq)[-third_perc:-second_perc]] = fun_exp[-3]
        factor[np.argsort(freq)[:-third_perc]] = fun_exp[-4]
        #factor[np.nonzero(factor <= stats["25%"].values[0])] = fun_exp[0]
        #factor[np.nonzero(np.logical_and(factor > stats["25%"].values[0], factor <= stats["50%"].values[0]))] = fun_exp[1]
        #factor[np.nonzero(np.logical_and(factor > stats["50%"].values[0], factor <= stats["75%"].values[0]))] = fun_exp[2]
        #factor[np.nonzero(np.logical_and(factor > stats["75%"].values[0], factor < 1))] = fun_exp[3]j
        
        for j in range(array[i].shape[0]):
            cell = array[i][j].copy()
            factor_j = factor[np.where(pos == cell)[0]]
            array[i][j] = mv_cell(cell, x*factor_j, y*factor_j)
            
    return array

In [190]:
def anonymize_pfipf(array):
    array = array.copy()
    pfipf = compute_pfipf(array, number_people=2000, upper_bound=1024)
    
    for i in tqdm(range(array.shape[0])):
        x = 0
        y = 0
        while (x == 0) and (y == 0):
            x = np.random.randint(-1,2)
            y = np.random.randint(-1,2)
            
        first_perc = (int(np.count_nonzero(pfipf[i])*0.02)) or 20
        second_perc = (int(np.count_nonzero(pfipf[i])*0.03)) + first_perc
        third_perc = (int(np.count_nonzero(pfipf[i])*0.25)) + second_perc

        factor = pfipf[i].copy()
        factor[np.argsort(pfipf[i])[-first_perc:]] = fun_exp_pfipf[-1]
        factor[np.argsort(pfipf[i])[-second_perc:-first_perc]] = fun_exp_pfipf[-2]
        factor[np.argsort(pfipf[i])[-third_perc:-second_perc]] = fun_exp_pfipf[-3]
        factor[np.argsort(pfipf[i])[:-third_perc]] = fun_exp_pfipf[-4]            
        
        for j in range(array[i].shape[0]):
                cell = array[i][j].copy()
                factor_j = factor[cell]
                array[i][j] = mv_cell(cell, x*factor_j, y*factor_j)

    return array

In [191]:
ano_org_cell = anonymize_pfipf(org_cell)

#ano_org_cell = anonymize(org_cell)

100%|██████████| 2000/2000 [00:08<00:00, 231.92it/s]


In [192]:
df_ano = pd.DataFrame(ano_org_cell)
df_ano = df_ano.reset_index()
df_ano["index"] = df_ano.index + 1
df_ano = df_ano.melt("index").sort_values(["index", "variable"]).reset_index(drop=True).drop("variable", axis=1)
df_ano = df_ano.rename(columns={"index": "user_id", "value": "reg_id"})

In [193]:
df_ano.reg_id.to_csv("anonymized.csv", index=False, header="reg_id")